In [17]:
import pandas as pd
import numpy as np

In [18]:
barometric = pd.read_csv("barometric.csv", names=["row","unix_time", "date", "time", "barometric", "unnamed"])
direction = pd.read_csv("direction.csv", names=["row", "unix_time", "date", "time", "direction", "unnamed"])
humidity = pd.read_csv("humidity.csv", names=["row", "unix_time", "date", "time", "humidity", "unnamed"])
radiation = pd.read_csv("radiation.csv", names=["row", "unix_time", "date", "time", "radiation", "unnamed"])
speed = pd.read_csv("speed.csv", names=["row", "unix_time", "date", "time", "speed", "unnamed"])
# sunrise = pd.read_csv("sunrise.csv", names=["row", "unix_time", "date", "time", "sunrise", "unnamed"])
# sunset = pd.read_csv("sunset.csv", names=["row", "unix_time", "date", "time", "sunset", "unnamed"])
temperature = pd.read_csv("temperature.csv", names=["row", "unix_time", "date", "time", "temp", "unnamed"])

In [19]:
print barometric.size
print direction.size
print humidity.size
print radiation.size
print speed.size
# print sunrise.size
# print sunset.shape
print temperature.shape

198798
198798
198798
198798
198798
(33133, 6)


In [20]:
features = [barometric, direction, humidity, radiation, speed, temperature]
for feature in features:
    feature.drop(feature.columns[[0,2,3,5]], axis=1, inplace=True)
# feature = reduce(lambda left, right: pd.merge(left, right,on="unix_time"), features)
# features = iter(features)
# feature = next(features)
# for df_ in features:
#     feature = feature.merge(df_, on='unix_time')
# fea = pd.merge(barometric,sunrise, on="unix_time")
# fea.count(axis=0)
# feature.count(axis=0)

In [21]:
fea = features[0]
for feat in features[1:]:
    fea= pd.merge(fea,feat, on="unix_time", how="outer")
fea.count(axis=0)    

unix_time     33133
barometric    33133
direction     33133
humidity      33133
radiation     33133
speed         33133
temp          33133
dtype: int64

In [ ]:
fea.head()

In [ ]:
fea.shape

In [22]:
Y = fea["radiation"]
print Y.head()
X =fea[["barometric", "humidity", "speed", "temp", "direction"]]
print X.head()

0    1.27
1    1.25
2    1.25
3    1.25
4    1.27
Name: radiation, dtype: float64
   barometric  humidity  speed  temp  direction
0       30.46        80   7.87    48     151.95
1       30.46        79   7.87    48     176.14
2       30.46        79   9.00    48     177.54
3       30.46        79  13.50    48     167.56
4       30.47        79  15.75    48     177.81


In [23]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
# import xgboost as xgb
# xgbmat = xgb.DMatrix(X_train,y_train)
# params = {"objective": "reg:linear", "booster":"gblinear"}
# gbm = xgboost.XGBRegressor()
# gbm = xgb.train(dtrain=xgbmat,params=params)
# testdmat = xgb.DMatrix(X_test)
# y_pred = gbm.predict(testdmat)
# accuracy = gbm.score(y_pred, y_test)
# print accuracy

In [ ]:
# from sklearn.linear_model import LinearRegression
# clf = LinearRegression()
# clf.fit(X_train, y_train)
# print clf.score(clf.predict(X_test), y_test)

In [24]:
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error
rng = np.random.RandomState(31337)
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
xgb_model = xgb.XGBRegressor()
clf = GridSearchCV(xgb_model,
                   {'max_depth': [2,4,6],
                    'n_estimators': [50,100,200]}, verbose=1)
clf.fit(X,Y)
print(clf.best_score_)
print(clf.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   22.8s finished


0.611226710977
{'n_estimators': 50, 'max_depth': 4}


In [25]:
import pickle

pickle.dump(clf, open("best_radiation.pkl", "wb"))
clf2 = pickle.load(open("best_radiation.pkl", "rb"))
print(np.allclose(clf.predict(X), clf2.predict(X)))

True


In [26]:
X= pd.Series({"barometric": 300, "humidity": 0.5,
                  "speed": 203, "temp": 100, "direction": 315})
# X = np.array([300,0.5,203,100,300,302])
print X.shape

(5,)


In [27]:
clf2.predict(X)

array([ 489.89715576], dtype=float32)